### __How to use ArcgGIS API for python to build an ArcGIS Dashboard__

This notebook is a walk through of how to use ArcgGIS API for python to connect to a ArcGIS online geodatabase and build a ArcGIS Dashboard. The data used in this tutorial derived from Hudson River Park's community compost data. The purpose of this dashboard is to visualize the sucess of the composting program throughout the park and to incentivize locals to contribute to a compost bin in their community.  

In [25]:
from arcgis import GIS
import pandas as pd
from datetime import datetime
import json

In [26]:
# connect to ArcGIS Online using your company's username
portal = "https://hrpt.maps.arcgis.com/"
user = "company_username"
gis = GIS( profile="learn_user")

In [27]:
# source layer that has the live, un-summarized data
source_live_layer_item_id = '80fcd22ad0fd4e419512869738929740'
source_live_layer = gis.content.get(source_live_layer_item_id)
flayer = source_live_layer.layers[0]
flayer

<FeatureLayer url:"https://services.arcgis.com/4oSdFB1Bw9eBgXJz/arcgis/rest/services/Cartegraph_Tasks/FeatureServer/0">

##### Recalculate the year, month name, and month number for each row in the live layer
Step 1

In [28]:
# Name of the datetime field we will use to parse out the needed date elements
source_date_field_name = 'StopDateActual'

# out fields to bring back in our query
out_fields = ['OBJECTID', source_date_field_name, 'Year_Date', 'Month_Name', 'Month_Num']

# query for all the features in the live layer
res = flayer.query(where="1=1", out_fields=out_fields, return_geometry=False)

# loop through each feature, and parse out the date information we need
# building a list of edits along the way
# Note: res is short for result
edits = []
for f in res:
  # get the object id that is needed to update the correct feature
  oid = f.attributes['OBJECTID']
  
  # get the datetime value from the feature stored in milliseconds
  dte_ms = f.attributes[source_date_field_name]

  # convert the milliseconds to a datetime object so we can parse out the year and month
  dte_obj = datetime.fromtimestamp(dte_ms/1000)
  dte_year = dte_obj.year
  dte_month = dte_obj.month
  dte_month_name = dte_obj.strftime('%B')
  

  edit = {
    'attributes': {
      'OBJECTID': oid,
      'Year_Date': dte_year,
      'Month_Name': dte_month_name,
      'Month_Num': dte_month
    }
  }

  # add the edit to the list of edits
  edits.append(edit)

print(f'{len(edits)} edits ready to update')

3704 edits ready to update


##### Step 2 edit the live layer so that it now has the year, month number, and month name

**Note:** Be sure to check through each result to see if there were any errors.

In [29]:
edit_res = flayer.edit_features(updates=edits)

for er in edit_res['updateResults']:
  if er['success'] != True:
    print(f'error in updating feature with OBJECTID: {er["objectId"]}')
    print(er)

print('done updating live layer')

done updating live layer


##### This is the layer that will hold the live, summarized data #Compost_flayer_sum

In [30]:
source_summarized_layer_item_id = 'ee0a7e467b564afca982cb958503173c' 
source_summarized_layer = gis.content.get(source_summarized_layer_item_id)
flayer_summarized = source_summarized_layer.layers[0]
flayer_summarized

<FeatureLayer url:"https://services.arcgis.com/4oSdFB1Bw9eBgXJz/arcgis/rest/services/Compost_flayer_sum/FeatureServer/0">

##### Now run a query for all locations' geometry and drop_duplicates to get unique list of locations and their geometry.

**Note:** The data used in this tutorial is derived from an ArcGIS online geodatabase that is 
connected to a cartegraph operations managment system. This system is where data about an asset (feature)
is entered by a field team member.Therefore,'cgAssetID' is the compost bin asset number associated with a arcgis online feature. The information about this asset/ bin number is continuously populated by a field team member 
and is associated with a point.

In [31]:
locations_res = flayer.query(where='1=1', out_fields='cgAssetID')
unique_locations = locations_res.sdf.drop_duplicates(subset=['cgAssetID'], inplace=False)

##### Helper function to get the geometry of a location by LocationDescription value.

In [32]:
def get_location_geometry(location_name):
  shape_json = unique_locations.loc[unique_locations['cgAssetID'] == location_name, 'SHAPE'].iloc[0]
  return shape_json.JSON

##### Query for summarized statistics for each location by year, by month

In [33]:
out_statistics = [
   {
      'statisticType': 'sum',
      'onStatisticField': 'FoodScraps_float',
      'outStatisticFieldName': 'FoodScraps_float_sum'
   }
]

group_by_fields = 'cgAssetID, LocationDescription, Year_Date, Month_num, Month_Name'

res = flayer.query(where='1=1', out_statistics=out_statistics, group_by_fields_for_statistics=group_by_fields)
print(f'successfully summarized query into {len(res)} features')

successfully summarized query into 412 features


##### Combine attributes with geometry.

_Note: you need to specify geom as a json object like this json.load(geom) for it to show up on the map._
_If it's a string than loads and if it's a bianary file then use load:_ 
_https://stackoverflow.com/questions/11174024/attributeerrorstr-object-has-no-attribute-read_

In [34]:
# 
features = []
for f in res:
  feature = f.as_dict
  loc_name = feature['attributes']['cgAssetID']
  geom = get_location_geometry(loc_name)
  if geom is None:
    print (f'unable to get geometry for {loc_name} ... continuing.')
    continue
    
  # feature['geometry'] = geom

  feature['geometry'] = json.loads(geom)  
  features.append(feature)

print(f'successfully combined {len(features)} features with geometry to update')



successfully combined 412 features with geometry to update


##### Delete existing features.

In [35]:
trunc_res = flayer_summarized.manager.truncate()
trunc_res

{'success': True}

##### Add in our new features.

In [36]:
adds_res = flayer_summarized.edit_features(adds=features)
adds_res

{'addResults': [{'objectId': 1,
   'uniqueId': 1,
   'globalId': None,
   'success': True},
  {'objectId': 2, 'uniqueId': 2, 'globalId': None, 'success': True},
  {'objectId': 3, 'uniqueId': 3, 'globalId': None, 'success': True},
  {'objectId': 4, 'uniqueId': 4, 'globalId': None, 'success': True},
  {'objectId': 5, 'uniqueId': 5, 'globalId': None, 'success': True},
  {'objectId': 6, 'uniqueId': 6, 'globalId': None, 'success': True},
  {'objectId': 7, 'uniqueId': 7, 'globalId': None, 'success': True},
  {'objectId': 8, 'uniqueId': 8, 'globalId': None, 'success': True},
  {'objectId': 9, 'uniqueId': 9, 'globalId': None, 'success': True},
  {'objectId': 10, 'uniqueId': 10, 'globalId': None, 'success': True},
  {'objectId': 11, 'uniqueId': 11, 'globalId': None, 'success': True},
  {'objectId': 12, 'uniqueId': 12, 'globalId': None, 'success': True},
  {'objectId': 13, 'uniqueId': 13, 'globalId': None, 'success': True},
  {'objectId': 14, 'uniqueId': 14, 'globalId': None, 'success': True},
 